<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>
# 分类 Classification

分类这块内容主要就是决策树模型、朴素贝叶斯、神经网络、SVM模型

分类分两步走：一是训练，二是预测

![](images/classification.jpg)

## Decision Tree

决策树是一种类似于流程图的树结构，每个**内部节点**表示在一个属性上的测试，每个**分支**表示该测试的一个输出，而每个叶节点表示一个类的分块（即输出结果）。树的顶层节点是**根**节点。

决策树算法的优势：不需要任何领域的先验知识或者参数设置，并且能够处理较大规模的高维数据。并且模型输出直观，能够被人理解，能够输出特征的重要性。如图：
![](images/Dtree1.jpg)

老式决策树算法有：ID3、C4.5、CART，其中CART是二叉决策树，其他两种是多叉树。算法的基本流程如下：

* 使用三个参数，来调用算法：$D, attribute\_list, Attribute\_selction\_method$，$D$是数据分区，$attribute\_list$是描述元组属性的列表，$Attribute\_selction\_method$ specifies a heuristic procedure for selecting the attribute that “best” discriminates the given tuples
according to class（选择最好的划分节点），该过程使用属性选择度量（信息增益，Gini指数）。**树结构是否是严格的二叉树由属性选择度量决定。**
基尼指数强制结果树是二叉树。其他的度量允许多路划分。
* 树从单个节点N开始，N表示D中的训练元组。
* 若D中的元组都为同一类，则节点N变为树叶，并且用该类标记。
* 否则，调用算法$Attribute\_selction\_method$确定划分节点，指定分裂属性和分裂点（分裂子集），通常情况下，一个叶节点的数据都是尽可能**纯**的
* 节点$N$用分裂准则（$Attribute\_selction\_method$）标记作为节点上的测试。对分裂的每个输出，由节点N生长出一个分支。设A为分裂属性，根据训练结果$A$有$v$个不同取值$a_1, a_2, \cdots, a_v$。三种情况分别对应，a是离散的，b是连续的，c是离散的且必须是二叉树。

![](images/Dtree2.jpg)
* 对于$D$的每个结果分区$D_j$上的元组，递归形成决策树。
* 递归停止条件：
    * 分区$D$都属于同一类
    * 无剩余特征提供划分。达到最大深度。分裂点阀值不满足。
    * 分支无元组
* 返回决策树

算法如下：
![](images/Dtree_alg.jpg)

## 属性度量选择

### 信息增益 用于ID3
设节点N代表分区D的元组，选择具有最高信息增益的属性作为节点N的分裂属性，即划分该节点的孩子节点对父亲节点需要的信息量最小，并且返于这些分区中的最小随机性（不纯性）。使得对一个对象分类所需要的期望测试数目最小。

$$Info(D) = - \sum_{i=1}^{m} p_i log_2(p_i)$$

$p_i$是$D$中任意元组选择属于类$C_i$的非零概率，使用$\frac{|C_{i,D}|}{|D|}$来估计，$Info(D)$又称为$D$的熵。

此处使用$A$将$D$划分为$v$个子集，每个子集都去计算一遍，选择最大的$Gain$值

$$ Info_A(D) = \sum_{j=1}^{v} \frac{|D_j|}{|D|}\times Info(D_j) ~~~~ Gain(A) = Info(D) - Info_A(D)$$

其中$\frac{|D_j|}{|D|}$作为第j个分区的权重。期望信息越小，分区的纯度越高。（即最小化$Info_A(D)$）

**信息增益偏向大量值的属性**，即划分用户ID时，每个分区只包含一个元组，同时分裂的信息熵为0，虽然增益最大，但是对分类没有用。

### 增益率 用于C4.5
考虑信息的增益率，而不是数值增益。

$$ SplitInfo_A(D) = - \sum_{j=1}^{v} \frac{|D_j|}{|D|}\times log_2(\frac{|D_j|}{D}) ~~~~GainRate(A) = \frac{Gain(A)}{SplitInfo_A(D)} $$

相比增益数值，增益率有少许的改善，但是随着划分信息趋向于0，该比率就会变得不稳定。

### 基尼指数 用于CART
定义为：
$$Gini(D) = 1 - \sum_{i=1}^{m}p_i^2$$

$p_i$是$D$中任意元组选择属于类$C_i$的非零概率，使用$\frac{|C_{i,D}|}{|D|}$来估计，Gini指数只能划分为两个子集。同上，A作为划分点

$$ Gini_A(D) = \frac{|D_1|}{|D|}Gini(D_1) + \frac{|D_2|}{|D|}Gini(D_2) $$

选择最小基尼指数作为它的分裂子集。不纯度降低幅度：

$$ \Delta Gini(A) = Gini(D) - Gini_A(D) $$

**信息增益偏向于多值属性，信息增益率倾向于产生不平衡的划分，基尼指数偏向于多值属性，并且当类的数量很大时会有困难。**

## 树剪枝

先剪枝：在生成树的过程中，随时判断模型精度，精度下降，就剪掉。无法照顾全局。

后剪枝：在完全的生成树后，对数进行剪枝。时间复杂度比较高。

## iris花分类

In [1]:
from sklearn import datasets,metrics,tree
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
data_iris = datasets.load_iris()
data = data_iris['data']
label = data_iris['target']
X_train,X_val,Y_train,Y_val = train_test_split(data,label, test_size = 0.3, random_state = 0)
#-------------------------DecisionTreeClassifier - entropy
bay_model = tree.DecisionTreeClassifier(criterion = 'entropy')
bay_model.fit(X_train,Y_train)

preds = bay_model.predict(X_val)
print(bay_model)
print(metrics.f1_score(Y_val,preds,average = 'macro'))
print(classification_report(Y_val, preds))
print(tree.export_text(bay_model))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
0.975983436853002
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.94      0.97        18
           2       0.92      1.00      0.96        11

    accuracy                           0.98        45
   macro avg       0.97      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

|--- feature_2 <= 2.35
|   |--- class: 0
|--- feature_2 >  2.35
|   |--- feature_2 <= 4.95
|   |   |--- feature_3 <= 1.65
|   |   |   |--- class: 1
|   |   |--- feature_3 

In [3]:
#-------------------------DecisionTreeClassifier - gini
bay_model = tree.DecisionTreeClassifier(criterion = 'gini')
bay_model.fit(X_train,Y_train)

preds = bay_model.predict(X_val)
print(bay_model)
print(metrics.f1_score(Y_val,preds,average = 'macro'))
print(classification_report(Y_val, preds))
print(tree.export_text(bay_model))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
0.975983436853002
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.94      0.97        18
           2       0.92      1.00      0.96        11

    accuracy                           0.98        45
   macro avg       0.97      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

|--- feature_2 <= 2.35
|   |--- class: 0
|--- feature_2 >  2.35
|   |--- feature_2 <= 4.95
|   |   |--- feature_3 <= 1.65
|   |   |   |--- class: 1
|   |   |--- feature_3 >  

In [4]:
#-------------------------ExtraTreeClassifier - entropy
bay_model = tree.ExtraTreeClassifier(splitter = 'best', criterion='entropy')
bay_model.fit(X_train,Y_train)

preds = bay_model.predict(X_val)
print(bay_model)
print(metrics.f1_score(Y_val,preds,average = 'macro'))
print(classification_report(Y_val, preds))
print(tree.export_text(bay_model))

ExtraTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, random_state=None,
                    splitter='best')
0.9511784511784511
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.94      0.94      0.94        18
           2       0.91      0.91      0.91        11

    accuracy                           0.96        45
   macro avg       0.95      0.95      0.95        45
weighted avg       0.96      0.96      0.96        45

|--- feature_2 <= 2.35
|   |--- class: 0
|--- feature_2 >  2.35
|   |--- feature_0 <= 5.75
|   |   |--- feature_3 <= 1.50
|   |   |   |--- class: 1
|   |   |--- feature_3 >  1.50
|   |   |   |--- class: 2
|  

In [5]:
#-------------------------ExtraTreeClassifier - gini
bay_model = tree.ExtraTreeClassifier(splitter = 'best', criterion='gini')
bay_model.fit(X_train,Y_train)

preds = bay_model.predict(X_val)
print(bay_model)
print(metrics.f1_score(Y_val,preds,average = 'macro'))
print(classification_report(Y_val, preds))
print(tree.export_text(bay_model))

ExtraTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, random_state=None,
                    splitter='best')
0.9279503105590061
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.94      0.89      0.91        18
           2       0.83      0.91      0.87        11

    accuracy                           0.93        45
   macro avg       0.92      0.93      0.93        45
weighted avg       0.94      0.93      0.93        45

|--- feature_2 <= 2.35
|   |--- class: 0
|--- feature_2 >  2.35
|   |--- feature_2 <= 4.95
|   |   |--- feature_2 <= 4.75
|   |   |   |--- feature_2 <= 4.45
|   |   |   |   |--- class: 1
|   |   |   |--- feature